In [1]:
import logging
import random
import string
from datetime import datetime

import numpy as np
import dns.message, dns.query, dns.rdataclass, dns.rdatatype, dns.flags, dns.exception, dns.name
from tqdm import tqdm
import pandas as pd

IN = dns.rdataclass.from_text("IN")
NS = dns.rdatatype.from_text("NS")
SOA = dns.rdatatype.from_text("SOA")
DS = dns.rdatatype.from_text("DS")
A = dns.rdatatype.from_text("A")
AAAA = dns.rdatatype.from_text("AAAA")
RRSIG = dns.rdatatype.from_text("RRSIG")

In [2]:
num_domains = 10
num_query_repeat = 10

In [3]:
domains = [dns.name.from_text(f"{''.join(random.choices(string.ascii_lowercase, k=6))}.ml.adnssec.dedyn.io") for _ in range(num_domains)]
domains

[<DNS name ppkcpb.ml.adnssec.dedyn.io.>,
 <DNS name qugpvd.ml.adnssec.dedyn.io.>,
 <DNS name yaqjws.ml.adnssec.dedyn.io.>,
 <DNS name qhlhts.ml.adnssec.dedyn.io.>,
 <DNS name lnvlov.ml.adnssec.dedyn.io.>,
 <DNS name owgqjq.ml.adnssec.dedyn.io.>,
 <DNS name oiryjg.ml.adnssec.dedyn.io.>,
 <DNS name pjsskh.ml.adnssec.dedyn.io.>,
 <DNS name ifjtdr.ml.adnssec.dedyn.io.>,
 <DNS name srgmqs.ml.adnssec.dedyn.io.>]

In [4]:
open_resolvers = {row['Handle']: row['IPv4'] for _, row in pd.read_csv("traffic/open-resolvers.csv").iterrows()}
lab_resolvers = {row['Handle']: row['IPv4'] for _, row in pd.read_csv("traffic/lab-resolvers.csv").iterrows()}

In [5]:
resolvers = {
    **open_resolvers,
    #**lab_resolvers
}

In [6]:
resolvers

{'cisco-umbrella': '208.67.222.222',
 'cloudflare': '1.1.1.1',
 'comodo-secure-dns': '8.26.56.26',
 'cznic-odvr': '193.17.47.1',
 'freenom-world': '80.80.80.80',
 'google': '8.8.8.8',
 'neustar-free-recursive': '156.154.70.1',
 'norton-connectsafe': '199.85.126.10',
 'opennic': '194.36.144.87',
 'oracle-dyn': '216.146.35.35',
 'quad9': '9.9.9.9'}

In [7]:
def query(qname, resolver, cd, rdtype=A):
    q = dns.message.make_query(qname, A, want_dnssec=True)
    if cd:
        q.flags = q.flags | dns.flags.CD
    try:
        return dns.query.udp(q, where=resolver, timeout=2)
    except dns.exception.Timeout:
        return dns.query.udp(q, where=resolver, timeout=5)

    
def check_resolver(resolver, d, rname):
    base = {
        'qname': d.to_text(),
        'resolver': resolver,
        'time': datetime.now(),
        'handle': rname,
    }
    
    try:
        rdtype = A
        logging.debug(f'query {dns.rdatatype.to_text(rdtype)} {d} at {resolver}')

        try:
            r = query(d, resolver, cd=True)
        except dns.exception.Timeout:
            return {
                'status': 'timeout',
                **base,
            }

        base['rcode'] = r.rcode()
        if r.rcode() == 2:  # servfail
            return {
                'status': 'servfail',
                **base,
            }

        ad1 = 'AD' in dns.flags.to_text(r.flags)
        rrsigs = r.get_rrset(r.answer, d, IN, RRSIG, covers=A) or []
        if len(rrsigs) != 1:
            raise Exception(f"Expected exactly one signature on A/{d} when queried via {r}, but got {len(rrsigs)}: {rrsigs}")
        suite = rrsigs[0].algorithm
        ttl = rrsigs.ttl
        
        ad2 = None
        if not ad1:
            try:
                r = query(d, resolver, cd=False)
            except dns.exception.Timeout:
                return {
                    'status': 'timeout',
                    **base,
                }
            ad2 = 'AD' in dns.flags.to_text(r.flags)
            

        return {
            'algorithm': suite,
            'ttl': ttl,
            'ad1': ad1,
            'ad2': ad2,
            'status': 'ok',
            **base,
        }
    except Exception as e:
        e.rname = rname
        e.r = resolver
        e.d = d
        raise


In [8]:
import concurrent

executor = concurrent.futures.ThreadPoolExecutor(1)

In [9]:
def run_queries(resolvers):
    errors = []
    results = []    
    futures = [executor.submit(check_resolver, r, d, rname=rname) for _ in range(num_query_repeat) for d in domains for rname, r in resolvers.items()]
    with tqdm(total=len(futures), desc="Querying") as pbar:
        for future in concurrent.futures.as_completed(futures):
            pbar.update(1)
            if future.exception():
                logging.warning(f"{future.exception().r}: {future.exception()}")
                errors.append(future.exception().r)
            else:
                results.append(future.result())
    return results, errors

In [10]:
results, errors = run_queries(resolvers)

Querying: 100%|██████████| 1100/1100 [07:29<00:00,  2.45it/s]


In [11]:
len(results), len(errors)

(1100, 0)

In [12]:
print("error rate", len(errors) / len(resolvers))

error rate 0.0


In [13]:
data = pd.DataFrame(results)
data['secure'] = (data['ad1'] | data['ad2']).astype('float')
data.to_pickle(f'resolvers-data.pickle')
data

,algorithm,ttl,ad1,ad2,status,qname,resolver,time,handle,rcode,secure
0,15.0,3600.0,False,True,ok,ppkcpb.ml.adnssec.dedyn.io.,208.67.222.222,2021-09-08 15:48:51.829540,cisco-umbrella,0.0,1.0
1,15.0,3600.0,False,True,ok,ppkcpb.ml.adnssec.dedyn.io.,1.1.1.1,2021-09-08 15:48:52.091895,cloudflare,0.0,1.0
2,14.0,3600.0,True,None,ok,ppkcpb.ml.adnssec.dedyn.io.,8.26.56.26,2021-09-08 15:48:52.501014,comodo-secure-dns,0.0,1.0
3,10.0,1792.0,False,True,ok,ppkcpb.ml.adnssec.dedyn.io.,193.17.47.1,2021-09-08 15:48:52.710572,cznic-odvr,0.0,1.0
4,16.0,3600.0,False,False,ok,ppkcpb.ml.adnssec.dedyn.io.,80.80.80.80,2021-09-08 15:48:52.817791,freenom-world,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1095,14.0,3266.0,False,False,ok,srgmqs.ml.adnssec.dedyn.io.,156.154.70.1,2021-09-08 15:56:20.979433,neustar-free-recursive,0.0,0.0
1096,14.0,3208.0,False,False,ok,srgmqs.ml.adnssec.dedyn.io.,199.85.126.10,2021-09-08 15:56:21.030911,norton-connectsafe,0.0,0.0
1097,10.0,3208.0,False,False,ok,srgmqs.ml.adnssec.dedyn.io.,194.36.144.87,2021-09-08 15:56:21.086978,opennic,0.0,0.0
1098,14.0,3208.0,True,None,ok,srgmqs.ml.adnssec.dedyn.io.,216.146.35.35,2021-09-08 15:56:21.167609,oracle-dyn,0.0,1.0


In [29]:
ALGO_NAME = {
    5: 'rsasha1', 
    7: 'rsasha1nsec3sha1', 
    8: 'rsasha256', 
    10: 'rsasha512',
    13: 'ecdsap256sha256', 
    14: 'ecdsap384sha384', 
    15: 'ed25519', 
    16: 'ed448',
}

def values(s):
    hasna = np.isnan(s).any()
    values = set(ALGO_NAME.get(int(x), x) for x in s if not np.isnan(x))
    return values | {None} if hasna else values
    
def mean_na(s):
    return s.mean(skipna=False)

def status(s):
    m = s.mean()
    if m == 0:
        return 'zone unavailable'
    if m == 1:
        return 'zone secure'
    return 'mixed'

data.groupby(['resolver', 'handle', 'status'], dropna=False).agg({
    'status': ['count'],
    'secure': [status],  # add mean_na?
    'algorithm': [values],
}).sort_values([('secure', 'status'), 'handle']).style.apply(lambda row: ['color: red;' if row[('secure', 'status')] != 'zone secure' else 'color: darkgreen;'] * len(row), axis=1)

,,,status,secure,algorithm
,,,count,status,values
resolver,handle,status,,,
208.67.222.222,cisco-umbrella,ok,99,zone secure,{'ed25519'}
1.1.1.1,cloudflare,ok,100,zone secure,{'ed25519'}
8.26.56.26,comodo-secure-dns,ok,100,zone secure,{'ecdsap384sha384'}
193.17.47.1,cznic-odvr,ok,100,zone secure,{'rsasha512'}
8.8.8.8,google,ok,100,zone secure,{'ed25519'}
216.146.35.35,oracle-dyn,ok,100,zone secure,{'ecdsap384sha384'}
9.9.9.9,quad9,ok,54,zone secure,{'ed448'}
208.67.222.222,cisco-umbrella,servfail,1,zone unavailable,{None}


# Debug

In [15]:
if errors:
    logging.basicConfig(level=logging.DEBUG, force=True)
    check_resolver(errors_retry[0])